In [ ]:
import requests
from bs4 import BeautifulSoup
from google_play_scraper import Sort, reviews
import pandas as pd

In [8]:
url = "https://play.google.com"
item = "스도쿠"
link = f"https://play.google.com/store/search?q={item}&c=apps"

In [9]:
response = requests.get(link)
response.status_code

200

In [10]:
html = response.text
soup = BeautifulSoup(html, 'lxml')

In [11]:
list = soup.find('div', attrs ={'class':'fUEl2e NIkkXb'})

In [12]:
g_list = list.find_all('div', attrs = {'class':'ULeU3b'})

In [13]:
len(g_list)

30

In [14]:
link_list = []
app_ids = []

for i in g_list:
    g = i.find('a')['href']
    g_link = url + g
    link_list.append(g_link)
    
    ids = g.split('id=')[1]
    app_ids.append(ids)

In [15]:
len(link_list)

30

In [16]:
Top10 = link_list[:10]
app_ids_top10 = app_ids[:10]

In [17]:
result = []

for i in Top10:
    response = requests.get(i)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('span', attrs={'class':'AfwdI'}).text
    img = soup.select_one( 'div > img')['src']
    intro = soup.find('div', attrs= {'class':'bARER'}).text
    
    box = soup.find('div', attrs={'class':'w7Iutd'})
    grade = box.find_all('div', attrs={'class':'wVqUob'})[0].text.split('star')[0]
    d_num = box.find_all('div', attrs={'class':'wVqUob'})[1].text.split('+')[0]
    
    result.append({'title':title, 'img_link':img, 'intro':intro, 'rate':grade, 'download':d_num})
    
    

In [18]:
import pandas as pd

In [19]:
df1 = pd.DataFrame(result)
df2 = pd.DataFrame(app_ids_top10)
df3 = pd.concat([df1, df2], axis=1)

In [20]:
df3 = df3.rename(columns={0: "app_ids"})

In [36]:
def mask_username(name):
    if not isinstance(name, str):
        return name

    if len(name) <= 1:
        return "*"
    else:
        return name[0] + "*" * (len(name) - 1)

df_final = pd.DataFrame()

for idx, app_id in enumerate(app_ids_top10, start=1):

    review_list, _ = reviews(
        app_id,
        lang='ko',
        country='kr',
        sort=Sort.NEWEST,
        count=300
    )

    df = pd.DataFrame(review_list)
    df["app_ids"] = app_id
    print(len(df))

    if "userName" in df.columns:
        df["userName"] = df["userName"].apply(mask_username)

    df_final = pd.concat([df_final, df], axis=0)
    
    #file_name = f"reviews_sudoku{idx}.csv"
    #df.to_csv(file_name, index=False)

print("완료")

300
300
300
300
300
175
300
300
17
38
완료


In [37]:
merged_df = pd.merge(df_final, df3, on="app_ids", how="left")

In [42]:
merged_df.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'app_ids', 'title', 'img_link', 'intro',
       'rate', 'download'],
      dtype='object')

In [47]:
merged_df.to_csv('output_merge.csv', index=False)